In [ ]:

%load_ext autoreload
%autoreload 2



from jax import config
config.update("jax_enable_x64", True)
import jax
import json
from jax.tree_util import Partial


from dataclasses import dataclass
import warnings
from typing import List, Union

import cola

import jax.numpy as jnp
import jax.random as jr
from jaxtyping import (
    Array,
    Float,
    install_import_hook,
    Num,
)
import tensorflow_probability.substrates.jax.bijectors as tfb

#with install_import_hook("gpjax", "beartype.beartype"):
import gpjax as gpx
from gpjax.typing import (
    Array,
    ScalarInt,
    ScalarFloat,
)
from gpjax.distributions import GaussianDistribution

import matplotlib.pyplot as plt
from matplotlib import rcParams
plt.style.use(
    "https://raw.githubusercontent.com/JaxGaussianProcesses/GPJax/main/docs/examples/gpjax.mplstyle"
)
colors = rcParams["axes.prop_cycle"].by_key()["color"]



import gpjax as gpx
import jax.numpy as jnp
import jax.random as jr
import matplotlib.pyplot as plt
key = jr.PRNGKey(123)

import scipy.optimize as spo
from dataclasses import dataclass
from gpjax.base import param_field, static_field
import math
from gpjax.kernels.base import AbstractKernel
from gpjax.kernels import RBF
import optax as ox
from gpjax.kernels.stationary.utils import squared_distance
import tensorflow_probability.substrates.jax.bijectors as tfb
import tensorflow_probability.substrates.jax.distributions as tfd
from gpjax.typing import (
    Array,
    ScalarFloat,
)
from jaxtyping import (
    Float,
    Num,
)
from simple_pytree import Pytree
from optax import GradientTransformation
import jax.tree_util as jtu





The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:

def optim_builder(optim_pytree):

    def _init_leaf(o, p):
        if isinstance(o, GradientTransformation):
            return o.init(p)
        else:
            return None

    def _update_leaf(o, u, s, p):
        if isinstance(o, GradientTransformation):
            return tuple(o.update(u, s, p))
        else:
            return jtu.tree_map(jnp.zeros_like, p)

    def _get_updates(o, u, p):
        if isinstance(o, GradientTransformation):
            return u[0]
        else:
            return u
    
    def _get_state(o, u):
        if isinstance(o, GradientTransformation):
            return u[1]
        else:
            return None

    def init_fn(params):
        return jtu.tree_map(_init_leaf, optim_pytree, params, is_leaf=lambda x: isinstance(x, GradientTransformation))

    def update_fn(updates, state, params):
        updates_state = jtu.tree_map(_update_leaf, optim_pytree, updates, state, params, is_leaf=lambda x: isinstance(x, GradientTransformation))
        updates = jtu.tree_map(_get_updates, optim_pytree, updates_state, params, is_leaf=lambda x: isinstance(x, GradientTransformation))
        state = jtu.tree_map(_get_state, optim_pytree, updates_state, is_leaf=lambda x: isinstance(x, GradientTransformation))

        return updates, state

    return GradientTransformation(init_fn, update_fn)


# 100_000 entries sampled across time/lat/lon over first day of data

## X2D has:

0"Sea surface temperature (K)"

1"Sensible heat flux (W/m^2)"

2"Latent heat flux (W/m^2)"

3"Vertically-integrated moisture convergence (kg/m^2)"

4"Column relative humidity (%)"


## X3D has:

0"Absolute temperature (K)"

1"Relative humidity (%)"

2"Specific humidity (kg/kg)"

3"Geopotential height (m^2 s^-2)"

4"Zonal wind (m/s)"

5"Meridional wind (m/s)"

6"Potential temperature (K)"

7"Equivalent potential temperature (K)"

8"Equivalent potential temperature saturation deficit (K)"

9"Saturated equivalent potential temperature (K)"

10"MSE-conserving plume buoyancy (m/s^2)"


## static has:

0"Land-sea mask"

1"Angle of sub-gridscale orography (rad)Anisotropy of sub-gridscale orography"

2"Standard deviation of sub-gridscale orography"

3"Slope of sub-gridscale orography"

## Y has:

0"ERA5 Precipitation (mm/hr)"

1"TRMM Precipitation (mm/hr)"

2"TRMM Relative Error (%)"

# plev are
1000.,   2000.,   3000.,   5000.,   7000.,  10000., 15000.,
20000.,  25000.,  30000.,  40000.,  50000.,  60000.,  70000.,80000.,  85000.,  90000.,  92500.,  95000.,  97500., 100000.


In [ ]:
# load data
X2d = jnp.array(jnp.load("..//data/100_000_one_day/X2D_sample.npy"), dtype=jnp.float64) # [N, D]
X3d = jnp.array(jnp.load("../data/100_000_one_day/X3D_sample.npy"), dtype=jnp.float64) # [N, D]
Xstatic = jnp.array(jnp.load("../data/100_000_one_day/XSTATIC_sample.npy"), dtype=jnp.float64) # [N, D]
Y = jnp.array(jnp.load("../data/100_000_one_day/Y_sample.npy"), dtype=jnp.float64) # [N, 1]
pressure = jnp.array([[1000,2000,3000,5000,7000,10000,15000,20000,25000,30000,40000,50000,60000,70000,80000, 85000,90000,92500,95000,97500,100000]], dtype=jnp.float64)


# random shuffle
X2d = jr.permutation(key, X2d)
X3d = jr.permutation(key, X3d)
Xstatic = jr.permutation(key, Xstatic)
Y = jr.permutation(key, Y)

# look at ERA5 rain
Y = Y[:,0:1]  

# remove any entries with nan
X3d_nan_idx = jnp.isnan(X3d).any(axis=1).any(axis=1)
X2d_nan_idx = jnp.isnan(X2d).any(axis=1)
Xstatic_nan_idx = jnp.isnan(Xstatic).any(axis=1)
Y_nan_idx = jnp.isnan(Y).any(axis=1)
any_nan = X3d_nan_idx | X2d_nan_idx | Y_nan_idx | Xstatic_nan_idx
no_nan = ~ any_nan
print(f"Removed {any_nan.sum()} entries with nan")
X2d = X2d[no_nan,:]
X3d = X3d[no_nan,:,:]
Xstatic = Xstatic[no_nan,:]
Y = Y[no_nan,:]


# just keep the "prognostic" 3d inputs that joe considered (for now)
# RH, tehta_e^+, theta_e, theta_e^*
names_3d =  ["RH", "tehta_e+", "theta_e", "theta_e*"]
idx_3d = [1, 9, 7, 8 ]
X3d = X3d[:,idx_3d,:]


# # also use his "normalisatopm" for sigma_o
sigma_o = jnp.where(Xstatic[:,0]<0.5, 0.0, 1.0+jnp.log(1+Xstatic[:,2]))
Xstatic = Xstatic.at[:,2].set(sigma_o)
idx_static = [2]
Xstatic = Xstatic[:,idx_static]


# no 2d for now
idx_2d = []
X2d = X2d[:,idx_2d]


#remove all pressure levels below 500 hPA
lowest_idx = 7 # 11
print(f"Removed all pressure levels below {pressure[:,lowest_idx]} hPa")
X3d = X3d[:, :, lowest_idx:]
pressure_levels = pressure[:,lowest_idx:]

# # remove no rain days
# print(f"Removed {(Y[:,0]==0).sum()} entries with zero rain")
# X3d = X3d[Y[:,0]>0,:]
# X2d = X2d[Y[:,0]>0,:]
# Xstatic = Xstatic[Y[:,0]>0,:]
# Y = Y[Y[:,0]>0,:]


# also log Y
print(f"Applied log transform to Y")
Y = jnp.log(Y+1e-12)
print(f"then standardized Y")
Y_mean = jnp.mean(Y)
Y_std = jnp.std(Y)
Y = (Y - Y_mean) / Y_std

# standardize inputs
X3d_std = jnp.std(X3d, axis=0)
X3d_mean = jnp.mean(X3d,axis=0)
X3d = (X3d - X3d_mean) / X3d_std
X2d_std = jnp.std(X2d, axis=0)
X2d_mean = jnp.mean(X2d,axis=0)
X2d = (X2d - X2d_mean) / X2d_std
Xstatic_std = jnp.std(Xstatic, axis=0)
Xstatic_mean = jnp.mean(Xstatic,axis=0)
Xstatic = (Xstatic - Xstatic_mean) / Xstatic_std


# look at all data but 1_000 test

N_test = 100
N_train = len(X2d) - N_test 
print(f"Using {N_train} training and {N_test} testing points!")
num_2d_variables= X2d.shape[1]
num_3d_variables= X3d.shape[1]
num_static_variables= Xstatic.shape[1]
num_not_3d_variables = num_2d_variables + num_static_variables
num_variables = num_2d_variables + num_3d_variables + num_static_variables
print(f"using {num_static_variables} static variables")
print(f"using {num_2d_variables} 2d variables")
print(f"using {num_3d_variables} 3d variables")






Removed 24779 entries with nan
Removed all pressure levels below [20000.] hPa
Applied log transform to Y
then standardized Y
Using 75121 training and 100 testing points!
using 1 static variables
using 0 2d variables
using 4 3d variables


In [ ]:
@dataclass
class VerticalSmoother(gpx.base.Module):
    smoother_mean: Float[Array, "1 D"]  = param_field(None)
    smoother_input_scale: Float[Array, "1 D"] = param_field(None, bijector=tfb.Softplus(low=jnp.array(1.0e-5, dtype=jnp.float64)))
    Z_levels: Float[Array, "1 L"] = static_field(pressure_levels)

    def __post_init__(self):
        self.Z_mean = jnp.mean(self.Z_levels)
        self.Z_std = jnp.std(self.Z_levels)
        self.Z_levels = (self.Z_levels - self.Z_mean) / self.Z_std

    def __call__(self,X3d: Num[Array, "N D L"]) -> Num[Array, "N D"]:
        smoothing_weights = jnp.exp(-0.5*((self.Z_levels-self.smoother_mean.T)/(self.smoother_input_scale.T))**2) # [D, L]
        smoothing_weights = (smoothing_weights/ jnp.sum(smoothing_weights, axis=-1, keepdims=True)) # [D, L]
        return jnp.sum(jnp.multiply(smoothing_weights , X3d), axis=-1) # [N, D]


@dataclass
class CustomDataset(Pytree):

    X2d: Num[Array, "N D"] = None
    X3d: Num[Array, "N D L"] = None
    Xstatic: Num[Array, "N D"] = None
    y: Num[Array, "N Q"] = None
    smoother: VerticalSmoother = None

    def __post_init__(self) -> None:
        X_temp = self.prep_X(self.X2d, self.X3d, self.Xstatic)
        gpx.dataset._check_shape(X_temp, self.y)
        gpx.dataset._check_precision(X_temp, self.y)

    @property
    def n(self) -> int:
        return self.X.shape[0]

    @property
    def in_dim(self) -> int:
        return self.X.shape[1]

    @property
    def X(self):
        return self.prep_X(self.X2d, self.X3d, self.Xstatic)

    def prep_X(self, X2d: Num[Array, "N D"], X3d: Num[Array, "N D L"], Xstatic: Num[Array, "N D"]):
        X3d_to_2d = self.smoother(X3d) # [N, D]
        return jnp.hstack((X3d_to_2d, X2d, Xstatic)) # always use 3d inputs first



@dataclass
class VerticallyIntegratedRBF(RBF):
    smoother: VerticalSmoother = None
    

def prep_kernel(num_variables: int, num_3d_variables, Y, bij_lengthscale:bool = True):
    lengthscale_bij = tfb.SoftClip(jnp.array(1e-2, dtype=jnp.float64),jnp.array(1e2, dtype=jnp.float64))
    smoother = VerticalSmoother(jnp.array([[0.0]*num_3d_variables]), jnp.array([[1.0]*num_3d_variables]))
    base_kernel = VerticallyIntegratedRBF(lengthscale=jnp.array([1.0]*num_variables), variance = jnp.var(Y), smoother = smoother)
    if bij_lengthscale:
        base_kernel = base_kernel.replace_bijector( lengthscale = lengthscale_bij)
    return base_kernel


def plot_params(model, title=None):
    if isinstance(model, gpx.variational_families.AbstractVariationalFamily):
        model = model.posterior
    plt.figure()
    opt_3d_kernel =model.prior.kernel
    lengthscales = opt_3d_kernel.lengthscale
    smoother = model.prior.kernel.smoother
    #smoothing_weights = opt_3d_kernel.smoother_bias.T + (1 / (jnp.sqrt(2*math.pi)*opt_3d_kernel.smoother_input_scale.T))*jnp.exp(-0.5*(opt_3d_kernel.Z_levels-opt_3d_kernel.smoother_mean.T)**2/(opt_3d_kernel.smoother_input_scale).T) # [4, 21]
    #smoothing_weights = (jnp.exp(-0.5*((opt_3d_kernel.Z_levels-opt_3d_kernel.smoother_mean.T)/(opt_3d_kernel.smoother_input_scale.T))**2)) # [4, 21]
    smoothing_weights =  jnp.exp(-0.5*((smoother.Z_levels-smoother.smoother_mean.T)/(smoother.smoother_input_scale.T))**2) # [4, 21]
    smoothing_weights = (smoothing_weights/ jnp.sum(smoothing_weights, axis=-1, keepdims=True)) # [4, 21]
    for i in range(num_3d_variables):
        plt.plot(smoothing_weights[i,:].T,pressure_levels[0,:], label=f"{names_3d[i]} with lengthscales {lengthscales[i]:.2f}")
    plt.legend()
    plt.title(title+f"other lengthscales are {lengthscales[num_3d_variables:]}")
    plt.gca().invert_yaxis()



In [ ]:
@dataclass
class VerticallyIntegratedRBF(VerticallyIntegratedRBF):
    dummy: AbstractKernel = gpx.kernels.RBF()

# prep fancy kernel for model
# first fit with small data to get init for SVGP

num_data_for_init=100

kernel = prep_kernel(num_variables, num_3d_variables, Y[:num_data_for_init,:])
D_small = CustomDataset(
    X2d = X2d[:num_data_for_init,:], 
    X3d = X3d[:num_data_for_init,:], 
    Xstatic = Xstatic[:num_data_for_init,:], 
    y = Y[:num_data_for_init,:],
    smoother = kernel.smoother,
)


prior = gpx.gps.Prior(mean_function= gpx.mean_functions.Zero(), kernel = kernel)
likelihood = gpx.likelihoods.Gaussian(num_datapoints=D_small.n)
posterior = prior * likelihood
objective = jax.jit(gpx.objectives.ConjugateMLL(negative=True))
#objective = jax.jit(gpx.objectives.NonConjugateMLL(negative=True))
opt_posterior, history = gpx.fit(
    model=posterior,
    objective=objective,
    train_data=D_small,
    optim=ox.adamw(learning_rate=1e-1),
    num_iters=100,
    key=jr.PRNGKey(42),
    safe=False,
)
plt.plot(history)
plot_params(opt_posterior,D_small,  title="initial fit with small data")



# # choose inducing inputs and init SVGP
# num_inducing = 100
# z = jr.normal(key, (num_inducing , D.X.shape[-1])) # allow this


# init_posterior_at_inducing = opt_posterior.predict(z, D_small)
# # todo try whitening ?
# q = gpx.variational_families.VariationalGaussian(
#     posterior=opt_posterior, 
#     inducing_inputs=z,
#     variational_mean= init_posterior_at_inducing.mean()[:,None],
#     variational_root_covariance= jnp.linalg.cholesky(init_posterior_at_inducing.covariance() + 1e-5 * jnp.eye(num_inducing)), # todo check this is right!
# )

# #q = q.replace_trainable(inducing_inputs=False)
# # q = q.replace_trainable(variational_mean=False)
# # q = q.replace_trainable(variational_root_covariance=False)

# objective = jax.jit(gpx.objectives.ELBO(negative=True))


# def zero_grads():
#     def init_fn(_): 
#         return ()
#     def update_fn(updates, state, params=None):
#         return jax.tree_map(jnp.zeros_like, updates), ()
#     return ox.GradientTransformation(init_fn, update_fn)

# optim_pytree = q.replace(
#     posterior = zero_grads(), 
#     variational_mean = ox.adamw(1e-2),
#     variational_root_covariance = ox.adamw(1e-2), 
#     inducing_inputs= ox.adamw(1e-2),
#     )





# optim = optim_builder(optim_pytree)

# opt_q, history = gpx.fit(
#     model=q,
#     objective=objective,
#     train_data=D,
#     optim=optim,
#     num_iters=500,
#     key=jr.PRNGKey(42),
#     batch_size=512,
# )

# plt.figure()
# plt.plot(history)
# plot_params(opt_q,D,  title="full fit")


  0%|          | 0/100 [00:00<?, ?it/s]

TracerBoolConversionError: Attempted boolean conversion of traced array with shape bool[]..
The error occurred while tracing the function _body_fun at /home/henry/Documents/python_dev/GPJax/gpjax/scan.py:136 for scan. This value became a tracer due to JAX operations on these lines:

  operation a[35m:bool[][39m = eq b c
    from line <string>:4 (__eq__)
See https://jax.readthedocs.io/en/latest/errors.html#jax.errors.TracerBoolConversionError